# developing external testting tricks?

## the OpenAI/BigCode variant only works on UNix

## I know that tempdir and subprocess works and does timeout... but can you actually recover from that?
should I allow `wgpu-shadertoy` to take a code arg in cli? instead of just
this will only work for singlepass shaders just now.

we hope that wgpu 22 and get shader compilation info gets upstreamed to wgpu-py soon (and I will participate).

In [2]:
import os
import tempfile
import subprocess


file_template = """
from wgpu_shadertoy import Shadertoy

shader_code = '''{}'''

shader = Shadertoy(shader_code, shader_type="glsl", offscreen=True)

if __name__ == "__main__":
    shader.show()
    shader.snapshot(0.0)
"""

def run_shader_in_subprocess(shader_code, timeout=5):
    status = "ok" # default case
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False, encoding="utf-8") as f:
        f.write(file_template.format(shader_code))
        f.flush()
        try:
            p = subprocess.run(["python", f.name], capture_output=True, timeout=timeout)
            
        except subprocess.SubprocessError as e:
            if isinstance(e, subprocess.TimeoutExpired):
                status = "timeout"
            else:
                status = "error"
    # cleanup temp file, delete_on_close was only added in Python 3.12?
    os.remove(f.name)
        
    if status == "ok":
        if p.stderr != b"":
            status = "error"
    
    return status



In [1]:
new_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    // Time varying pixel color
    vec3 col = 0.5 + 0.5*cos(iTime+uv.xyx+vec3(0,2,4));

    // Output to screen
    fragColor = vec4(col,1.0);
}
"""


error_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord )
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    // Time varying pixel color
    vec3 col = 0.5 + 0.5*cos(iTime+uv.xyx+vec3(0,2,4));

    // Output to screen
    fragColor = vec4(coll,1.0);
}
"""

# this panics because it loses device
minimal_code = """
void mainImage( out vec4 fragColor, in vec2 fragCoord ) {

    vec3 col = vec3(0.0);
    float incr = 0.1;
    for (float i = 0.5; i < 3.0; i += max(0.0, iTime)) {
        col += vec3(0.2);
        // continue;
    }
    fragColor = vec4(col, 1.0);
}
"""

In [2]:
import multiprocessing
import time
from wgpu_shadertoy import Shadertoy

def minimal_run(shader_code, p_queue):
    print("running minimal run")
    try:
        # shader.show()
        shader = Shadertoy(shader_code, shader_type="glsl", offscreen=True)
        frame = shader.snapshot(10.0)
        del shader
        print(type(frame))
        p_queue.put("ok")
    except Exception as e:
        print(e)
        p_queue.put("error")

def run_as_thread(shader_code, timeout=5):
    # Use Manager().Queue() for better compatibility in some environments
    manager = multiprocessing.Manager()
    q = manager.Queue()
    print(f"running {q}")
    p = multiprocessing.Process(target=minimal_run, args=(shader_code, q))
    print(f"starting {p}")
    p.start()
    print(f"started {p}")
    # time.sleep(5)
    p.join(timeout)
    print(f"joined {p}")
    if p.is_alive():
        print(f"{p} is alive")
        p.terminate()
        print(f"{p} terminated")
        # p.join()
        return "timeout"
    else:
        print(f"{p} is not alive")
        p.terminate()
        print(f"{p} terminated for sure")
        p.join()
        print(f"{p} joined for sure")
    p.close()
    # print(f"returning {len(q)}")
    q_item = q.get_nowait()
    print(f"got {q_item}")
    # q.get_nowait() # clear this as well?

    try:
        return q.get()
    except Exception as e:
        print(e)
        return "error"
    finally:
        return "idk"

# Example usage
new_code = "your_shader_code_here"
print(run_as_thread(new_code))

running <queue.Queue object at 0x000001980AFA74C0>
starting <Process name='Process-2' parent=6304 initial>
started <Process name='Process-2' pid=3276 parent=6304 started>
joined <Process name='Process-2' pid=3276 parent=6304 stopped exitcode=1>
<Process name='Process-2' pid=3276 parent=6304 stopped exitcode=1> is not alive
<Process name='Process-2' pid=3276 parent=6304 stopped exitcode=1> terminated for sure
<Process name='Process-2' pid=3276 parent=6304 stopped exitcode=1> joined for sure


Empty: 

In [4]:
print(run_shader_in_subprocess(new_code, 10))
print(run_shader_in_subprocess(error_code, 10))
print(run_shader_in_subprocess(minimal_code, 10))

ok
error
timeout


In [5]:
from annotate import run_shader

print(run_shader(new_code)) # erroniously timesout because it's so slow...
print(run_shader(error_code))
print(run_shader(minimal_code))

timeout
error
timeout
